# Reserve Data Exploration
Extract and examine ENTSOE reserve price data from ZIP files


In [1]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
# Helper function to fetch data in chunks (avoid API limits)
def fetch_entsoe_in_chunks(base_url, start_date, end_date, chunk_days=15):
    """
    Fetch ENTSOE data in chunks to avoid API limits.
    
    Args:
        base_url: URL without periodStart/periodEnd parameters
        start_date: Start date string "YYYYMMDD0000"
        end_date: End date string "YYYYMMDD0000"
        chunk_days: Number of days per request (default 15)
    
    Returns:
        Combined response content (ZIP or XML)
    """
    from datetime import datetime, timedelta
    import time
    
    # Parse dates
    start = datetime.strptime(start_date, "%Y%m%d%H%M")
    end = datetime.strptime(end_date, "%Y%m%d%H%M")
    
    all_zips = []
    current = start
    
    while current < end:
        next_chunk = min(current + timedelta(days=chunk_days), end)
        
        period_start = current.strftime("%Y%m%d%H%M")
        period_end = next_chunk.strftime("%Y%m%d%H%M")
        
        url = f"{base_url}&periodStart={period_start}&periodEnd={period_end}"
        
        print(f"Fetching {period_start} to {period_end}...")
        
        response = requests.get(url)
        
        if response.status_code == 200:
            all_zips.append(response.content)
            print(f"  ✓ Got {len(response.content)} bytes")
        else:
            print(f"  ✗ Error {response.status_code}: {response.text[:200]}")
        
        current = next_chunk
        time.sleep(0.5)  # Rate limiting
    
    # Combine all ZIP files
    if len(all_zips) == 1:
        return all_zips[0]
    
    # Merge multiple ZIPs into one
    combined_buf = io.BytesIO()
    with zipfile.ZipFile(combined_buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
        for idx, zip_content in enumerate(all_zips):
            try:
                ztemp = zipfile.ZipFile(io.BytesIO(zip_content))
                for name in ztemp.namelist():
                    data = ztemp.read(name)
                    out_name = f"chunk{idx}_{name}"
                    zout.writestr(out_name, data)
            except zipfile.BadZipFile:
                # Might be plain XML, wrap it
                zout.writestr(f"chunk{idx}_response.xml", zip_content)
    
    combined_buf.seek(0)
    return combined_buf.read()

print("Helper function loaded")


Helper function loaded


In [3]:
# Process Type = aFRR/A51
# Market Agreement = Daily
# Fetch one week from each season in 2024
API_KEY = "3da11da4-cb1c-41ea-8f2e-e80e188b9e4b"

# Define one week from each season in 2024
# January (winter), April (spring), June (summer), October (fall)
weeks_2024 = [
    ("202501080000", "202501150000"),  # January week (Jan 8-15)
    ("202504080000", "202504150000"),  # April week (Apr 8-15)
    ("202506100000", "202506170000"),  # June week (Jun 10-17)
    ("202510070000", "202510140000"),  # October week (Oct 7-14)
]

base_url = (
    "https://web-api.tp.entsoe.eu/api?"
    "documentType=A81&"
    "businessType=B95&"
    "processType=A51&"
    "Type_MarketAgreement.Type=A01&"
    "controlArea_Domain=10YHU-MAVIR----U&"
    f"securityToken={API_KEY}"
)

print("Fetching aFRR Daily data for 4 seasonal weeks in 2025...")
all_zip_contents = []

for i, (start, end) in enumerate(weeks_2024, 1):
    print(f"\nWeek {i}/4: {start[:8]} to {end[:8]}")
    week_content = fetch_entsoe_in_chunks(base_url, start, end, chunk_days=7)
    all_zip_contents.append(week_content)
    print(f"  ✓ Week {i} data: {len(week_content)} bytes")

# Combine all week ZIP files into one
print("\nCombining all weeks...")
import zipfile

combined_buf = io.BytesIO()
with zipfile.ZipFile(combined_buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
    for idx, zip_content in enumerate(all_zip_contents):
        try:
            ztemp = zipfile.ZipFile(io.BytesIO(zip_content))
            for name in ztemp.namelist():
                data = ztemp.read(name)
                out_name = f"week{idx}_{name}"
                zout.writestr(out_name, data)
        except zipfile.BadZipFile:
            # Might be plain XML, wrap it
            zout.writestr(f"week{idx}_response.xml", zip_content)

combined_buf.seek(0)
afrr_d1_content = combined_buf.read()

# Create a mock response object to maintain compatibility
class MockResponse:
    def __init__(self, content):
        self.content = content
        self.status_code = 200

response_afrr_d1 = MockResponse(afrr_d1_content)
print(f"✓ Total combined aFRR Daily data: {len(afrr_d1_content)} bytes")

Fetching aFRR Daily data for 4 seasonal weeks in 2025...

Week 1/4: 20250108 to 20250115
Fetching 202501080000 to 202501150000...
  ✓ Got 9746 bytes
  ✓ Week 1 data: 9746 bytes

Week 2/4: 20250408 to 20250415
Fetching 202504080000 to 202504150000...
  ✓ Got 10189 bytes
  ✓ Week 2 data: 10189 bytes

Week 3/4: 20250610 to 20250617
Fetching 202506100000 to 202506170000...
  ✓ Got 9651 bytes
  ✓ Week 3 data: 9651 bytes

Week 4/4: 20251007 to 20251014
Fetching 202510070000 to 202510140000...
  ✓ Got 25841 bytes
  ✓ Week 4 data: 25841 bytes

Combining all weeks...
✓ Total combined aFRR Daily data: 55277 bytes


In [4]:
responses = [response_afrr_d1]

In [5]:
# Extract ZIP file

import io, zipfile
from pathlib import Path


zip_file1 = zipfile.ZipFile(io.BytesIO(response_afrr_d1.content))



zips = [zip_file1]  # your ZipFile objects

combined_buf = io.BytesIO()
with zipfile.ZipFile(combined_buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
    for i, z in enumerate(zips):
        for name in z.namelist():
            data = z.read(name)
            out_name = f"{i}_{Path(name).name}"  # avoid name collisions
            zout.writestr(out_name, data)

combined_buf.seek(0)
zf = zipfile.ZipFile(combined_buf, 'r')   # merged zip
xml_files = zf.namelist()                 # flat list of all entries
print(f"Merged {len(xml_files)} XML files")

xml_files_afrr_d1 = zip_file1.namelist()                 # flat list of all entries




Merged 21 XML files


In [6]:
# Extract XML files into a single DataFrame (store raw XML text)
import re


rows = []

for xml_file in xml_files:
    print(f"\nProcessing {xml_file}...")

    # Read XML content and decode (handle BOM safely)
    xml_bytes = zf.read(xml_file)
    xml_text = xml_bytes.decode('utf-8-sig', errors='replace')

    # Split concatenated XML documents while preserving the declaration
    docs = [d for d in re.split(r'(?=<\?xml\s)', xml_text) if d.strip()]
    if not docs:
        docs = [xml_text]

    for i, doc in enumerate(docs):
        # Ensure declaration at start; if missing, add a minimal one
        cleaned = doc.lstrip()
        if not cleaned.startswith('<?xml'):
            cleaned = '<?xml version="1.0" encoding="UTF-8"?>\n' + cleaned

        name = f"{xml_file}_part_{i}" if len(docs) > 1 else xml_file
        rows.append({'file': name, 'xml': cleaned})

xml_docs_df = pd.DataFrame(rows)
print(f"Collected {len(xml_docs_df)} XML documents into xml_docs_df")
display(xml_docs_df.head())



Processing 0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202501072300-202501152300.xml...

Processing 0_week1_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202504072200-202504152200.xml...

Processing 0_week2_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202506092200-202506172200.xml...

Processing 0_week3_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202510062200-202510072200.xml...

Processing 0_week3_002-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202510072200-202510082200.xml...

Processing 0_week3_003-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202510112200-202510122200.xml...

Processing 0_week3_004-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202510062200-202510072200.xml...

Processing 0_week3_005-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202510112200-202510122200.xml...

Processing 0_week3_006-AMOUNT_AND_PRICE

,file,xml
0,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
1,0_week1_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
2,0_week2_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
3,0_week3_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
4,0_week3_002-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."


In [7]:
from lxml import etree as LET
print("Imported LET from lxml.etree")


Imported LET from lxml.etree


In [8]:
# Parse a single balancing XML into one DataFrame (metadata + all points)
from lxml import etree as LET
from pathlib import Path
import pandas as pd


def _lname(tag: str) -> str:
    return tag.split('}')[-1] if '}' in tag else tag


def parse_balancing_xml_to_df(xml_text: str) -> pd.DataFrame:
    root = LET.fromstring(xml_text.encode('utf-8'))

    # Document-level metadata (leaf nodes outside Point)
    meta = {}
    for el in root.iter():
        ln = _lname(el.tag)
        if ln == 'Point':
            continue
        # skip if any Point descendant
        if el.xpath('.//*[local-name()="Point"]'):
            continue
        if len(list(el)) == 0:
            val = (el.text or '').strip()
            if val:
                key = f"doc.{ln}"
                # keep first occurrence
                if key not in meta:
                    meta[key] = val
    # include root attributes
    for k, v in root.attrib.items():
        meta[f"doc.@{k}"] = v

    rows = []
    # Iterate TimeSeries → Period → Point
    for ts in root.xpath('.//*[local-name()="TimeSeries"]'):
        ts_meta = {}
        for child in ts:
            ln = _lname(child.tag)
            if ln in ('Period', 'timeInterval'):
                continue
            # capture simple leaf text under TS (one level)
            if len(list(child)) == 0:
                ts_meta[f"ts.{ln}"] = (child.text or '').strip()
        for per in ts.xpath('.//*[local-name()="Period"]'):
            # period context
            start = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="start"]/text()')
            end = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="end"]/text()')
            res = per.xpath('.//*[local-name()="resolution"]/text()')
            per_meta = {
                'per.start': start[0] if start else None,
                'per.end': end[0] if end else None,
                'per.resolution': res[0] if res else None,
            }
            for pt in per.xpath('.//*[local-name()="Point"]'):
                row = {}
                # Point fields (flatten one level deep)
                for ch in list(pt):
                    ln = _lname(ch.tag)
                    if len(list(ch)) == 0:
                        row[f"pt.{ln}"] = (ch.text or '').strip()
                    else:
                        # flatten nested elements under Point one level
                        for sub in ch:
                            row[f"pt.{ln}.{_lname(sub.tag)}"] = (sub.text or '').strip()
                # Merge: document meta + TS meta + period meta + point
                full = {**meta, **ts_meta, **per_meta, **row}
                rows.append(full)
    return pd.DataFrame(rows)

# Source XML: prefer first entry in zf/xml_files, else fallback to example file
if 'zf' in globals() and 'xml_files' in globals() and xml_files:
    xml_text_src = zf.read(xml_files[0]).decode('utf-8-sig', errors='replace')
else:
    example_path = Path('notebooks/ActivationPriceXML.txt')
    if not example_path.exists():
        example_path = Path('../notebooks/ActivationPriceXML.txt')
    xml_text_src = example_path.read_text(encoding='utf-8')

points_full_df = parse_balancing_xml_to_df(xml_text_src)
print(points_full_df.shape)
display(points_full_df.head())


(990, 39)


,doc.mRID,doc.revisionNumber,doc.type,doc.process.processType,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.area_Domain.mRID,...,ts.currency_Unit.name,ts.quantity_Measure_Unit.name,ts.curveType,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,pt.imbalance_Price.category
0,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,1,55,101,A06
1,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,5,55,103,A06
2,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,9,55,105,A06
3,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,13,55,100,A06
4,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,21,55,114,A06


In [9]:
# Parse all XML documents in xml_docs_df into one DataFrame (metadata + points)
import pandas as pd

if 'xml_docs_df' not in globals() or xml_docs_df.empty:
    raise RuntimeError('xml_docs_df is empty or not defined. Build it first.')

all_rows = []
for _, r in xml_docs_df.iterrows():
    try:
        dfi = parse_balancing_xml_to_df(str(r['xml']))
        dfi['file'] = r.get('file', None)
        all_rows.append(dfi)
    except Exception as e:
        print(f"Skip {r.get('file', '?')}: {e}")

points_full_df = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()
print(f"points_full_df combined: shape={points_full_df.shape}, columns={len(points_full_df.columns)}")
display(points_full_df.head())


points_full_df combined: shape=(3707, 43), columns=43


,doc.mRID,doc.revisionNumber,doc.type,doc.process.processType,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.area_Domain.mRID,...,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,pt.imbalance_Price.category,file,doc.allocationDecision_DateAndOrTime.dateTime,doc.original_MarketProduct.marketProductType,ts.original_MarketProduct.marketProductType
0,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,2025-01-08T23:00Z,PT15M,1,55,101,A06,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,NaN,NaN,NaN
1,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,2025-01-08T23:00Z,PT15M,5,55,103,A06,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,NaN,NaN,NaN
2,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,2025-01-08T23:00Z,PT15M,9,55,105,A06,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,NaN,NaN,NaN
3,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,2025-01-08T23:00Z,PT15M,13,55,100,A06,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,NaN,NaN,NaN
4,ea3dd3ffa26347918ccced6fdb146ec9,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-12-11T15:41:38Z,10YHU-MAVIR----U,...,2025-01-08T23:00Z,PT15M,21,55,114,A06,0_week0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCIN...,NaN,NaN,NaN


In [10]:
points_full_df.columns

Index(['doc.mRID', 'doc.revisionNumber', 'doc.type', 'doc.process.processType',
       'doc.sender_MarketParticipant.mRID',
       'doc.sender_MarketParticipant.marketRole.type',
       'doc.receiver_MarketParticipant.mRID',
       'doc.receiver_MarketParticipant.marketRole.type', 'doc.createdDateTime',
       'doc.area_Domain.mRID', 'doc.start', 'doc.end', 'doc.businessType',
       'doc.type_MarketAgreement.type', 'doc.mktPSRType.psrType',
       'doc.flowDirection.direction', 'doc.currency_Unit.name',
       'doc.quantity_Measure_Unit.name', 'doc.curveType', 'doc.resolution',
       'doc.position', 'doc.quantity', 'doc.procurement_Price.amount',
       'doc.imbalance_Price.category', 'ts.mRID', 'ts.businessType',
       'ts.type_MarketAgreement.type', 'ts.mktPSRType.psrType',
       'ts.flowDirection.direction', 'ts.currency_Unit.name',
       'ts.quantity_Measure_Unit.name', 'ts.curveType', 'per.start', 'per.end',
       'per.resolution', 'pt.position', 'pt.quantity',
       'pt.pr

In [11]:
unique_counts = points_full_df.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (43, 1)


,n_unique
pt.procurement_Price.amount,1614
pt.quantity,180
ts.mRID,69
per.end,32
per.start,32
pt.position,24
file,21
doc.mRID,21
doc.allocationDecision_DateAndOrTime.dateTime,17
doc.start,11


In [12]:
points_full_df =  points_full_df[[
    "doc.process.processType",
    "ts.type_MarketAgreement.type",
    "ts.mktPSRType.psrType",
    "ts.flowDirection.direction",
    "per.start",
    "per.end",
    "per.resolution",
    "pt.position",
    "pt.quantity",
    "pt.procurement_Price.amount",
    "ts.currency_Unit.name",
]]


In [13]:
points_full_df

,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,1,55,101,HUF
1,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,5,55,103,HUF
2,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,9,55,105,HUF
3,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,13,55,100,HUF
4,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,21,55,114,HUF
...,...,...,...,...,...,...,...,...,...,...,...
3702,A51,A01,A05,A02,2025-10-12T22:00Z,2025-10-13T22:00Z,PT15M,33,1,250,HUF
3703,A51,A01,A05,A02,2025-10-12T22:00Z,2025-10-13T22:00Z,PT15M,37,1,164,HUF
3704,A51,A01,A05,A02,2025-10-12T22:00Z,2025-10-13T22:00Z,PT15M,77,0,0,HUF
3705,A51,A01,A05,A02,2025-10-12T22:00Z,2025-10-13T22:00Z,PT15M,81,2,30,HUF


In [14]:
# parse UTC 'Z' strings
start_utc = pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
end_utc   = pd.to_datetime(points_full_df['per.end'],   utc=True, errors='coerce')

# DST-aware Hungary time (Europe/Budapest)
points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Europe/Budapest')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Europe/Budapest')

points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Etc/GMT-2')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Etc/GMT-2')

points_full_df.head()

,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name,per.start_dt,per.end_dt
0,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,1,55,101,HUF,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00
1,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,5,55,103,HUF,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00
2,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,9,55,105,HUF,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00
3,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,13,55,100,HUF,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00
4,A51,A01,A03,A01,2025-01-07T23:00Z,2025-01-08T23:00Z,PT15M,21,55,114,HUF,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00


In [15]:
# Start of the day-period in Europe/Budapest
start_dt = (
    pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
      .dt.tz_convert('Europe/Budapest')
)

# Parse ISO 8601 resolution (PT15M/PT30M/PT60M/PT1H...) -> minutes
res_ex = points_full_df['per.resolution'].str.upper().str.extract(r'PT(\d+)([HM])')
mins = pd.to_numeric(res_ex[0], errors='coerce')
mins = np.where(res_ex[1].eq('H'), mins * 60, mins)
mins = pd.to_numeric(mins, errors='coerce')

# Positions are 1-based; compute offset and exact time
pos = pd.to_numeric(points_full_df['pt.position'], errors='coerce')
offset_min = (pos - 1) * mins

points_full_df['time_dt'] = start_dt + pd.to_timedelta(offset_min, unit='m')
points_full_df['time_end_dt'] = points_full_df['time_dt'] + pd.to_timedelta(mins, unit='m')

points_full_df[['per.start','per.resolution','pt.position','time_dt','time_end_dt']].head()

,per.start,per.resolution,pt.position,time_dt,time_end_dt
0,2025-01-07T23:00Z,PT15M,1,2025-01-08 00:00:00+01:00,2025-01-08 00:15:00+01:00
1,2025-01-07T23:00Z,PT15M,5,2025-01-08 01:00:00+01:00,2025-01-08 01:15:00+01:00
2,2025-01-07T23:00Z,PT15M,9,2025-01-08 02:00:00+01:00,2025-01-08 02:15:00+01:00
3,2025-01-07T23:00Z,PT15M,13,2025-01-08 03:00:00+01:00,2025-01-08 03:15:00+01:00
4,2025-01-07T23:00Z,PT15M,21,2025-01-08 05:00:00+01:00,2025-01-08 05:15:00+01:00


In [16]:
points_full_df =  points_full_df[[
    'per.start_dt', 
    'per.end_dt' ,
    'time_dt',
    'time_end_dt',
    'per.resolution',
    'pt.position', 
    "doc.process.processType",
    "ts.type_MarketAgreement.type",
    "ts.mktPSRType.psrType",
    "ts.flowDirection.direction",
    "pt.quantity",
    "pt.procurement_Price.amount",
    "ts.currency_Unit.name",
]]

In [17]:
points_full_df.head()

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 00:00:00+01:00,2025-01-08 00:15:00+01:00,PT15M,1,A51,A01,A03,A01,55,101,HUF
1,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 01:00:00+01:00,2025-01-08 01:15:00+01:00,PT15M,5,A51,A01,A03,A01,55,103,HUF
2,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 02:00:00+01:00,2025-01-08 02:15:00+01:00,PT15M,9,A51,A01,A03,A01,55,105,HUF
3,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 03:00:00+01:00,2025-01-08 03:15:00+01:00,PT15M,13,A51,A01,A03,A01,55,100,HUF
4,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 05:00:00+01:00,2025-01-08 05:15:00+01:00,PT15M,21,A51,A01,A03,A01,55,114,HUF


In [18]:
unique_counts = points_full_df.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (13, 1)


,n_unique
pt.procurement_Price.amount,1614
time_end_dt,756
time_dt,756
pt.quantity,180
per.start_dt,32
per.end_dt,32
pt.position,24
ts.mktPSRType.psrType,3
ts.flowDirection.direction,2
per.resolution,1


In [19]:
points_full_df_test = points_full_df[
    (points_full_df["doc.process.processType"] == "A51")
    & (points_full_df["ts.type_MarketAgreement.type"] == "A01")
    & (points_full_df["ts.mktPSRType.psrType"] == "A03")
    ] 
# aFrr and BESS & daily agreement

In [20]:
points_full_df_test.shape

(1289, 13)

In [21]:
points_full_df_test.groupby("ts.flowDirection.direction").count()

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
ts.flowDirection.direction,,,,,,,,,,,,
A01,690,690,690,690,690,690,690,690,690,690,690,690
A02,599,599,599,599,599,599,599,599,599,599,599,599


In [22]:
points_full_df_test.sort_values("time_dt", ascending=True).head(20)

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 00:00:00+01:00,2025-01-08 00:15:00+01:00,PT15M,1,A51,A01,A03,A01,55,101,HUF
21,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 00:00:00+01:00,2025-01-08 00:15:00+01:00,PT15M,1,A51,A01,A03,A02,0,0,HUF
1,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 01:00:00+01:00,2025-01-08 01:15:00+01:00,PT15M,5,A51,A01,A03,A01,55,103,HUF
2,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 02:00:00+01:00,2025-01-08 02:15:00+01:00,PT15M,9,A51,A01,A03,A01,55,105,HUF
3,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 03:00:00+01:00,2025-01-08 03:15:00+01:00,PT15M,13,A51,A01,A03,A01,55,100,HUF
4,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 05:00:00+01:00,2025-01-08 05:15:00+01:00,PT15M,21,A51,A01,A03,A01,55,114,HUF
5,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 06:00:00+01:00,2025-01-08 06:15:00+01:00,PT15M,25,A51,A01,A03,A01,46,149,HUF
22,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 06:00:00+01:00,2025-01-08 06:15:00+01:00,PT15M,25,A51,A01,A03,A02,43,180,HUF
23,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 07:00:00+01:00,2025-01-08 07:15:00+01:00,PT15M,29,A51,A01,A03,A02,17,511,HUF
6,2025-01-08 01:00:00+02:00,2025-01-09 01:00:00+02:00,2025-01-08 07:00:00+01:00,2025-01-08 07:15:00+01:00,PT15M,29,A51,A01,A03,A01,37,549,HUF


In [23]:
unique_counts = points_full_df_test.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (13, 1)


,n_unique
pt.procurement_Price.amount,855
time_end_dt,737
time_dt,737
pt.quantity,133
per.start_dt,32
per.end_dt,32
pt.position,24
ts.flowDirection.direction,2
per.resolution,1
ts.mktPSRType.psrType,1


In [24]:
points_full_df_test["pt.procurement_Price.amount"].astype(float).mean()

np.float64(1457.6710628394103)